# Chapter 8: Restructuring Data into Tidy Form
## Recipes
* [Tidying variable values as column names with stack](#Tidying-variable-values-as-column-names-with-stack)
* [Tidying variable values as column names with melt](#Tidying-variable-values-as-column-names-with-melt)
* [Stacking multiple groups of variables simultaneously](#Stacking-multiple-groups-of-variables-simultaneously)
* [Inverting stacked data](#Inverting-stacked-data)
* [Unstacking after a groupby aggregation](#Unstacking-after-a-groupby-aggregation)
* [Replicating pivot_table with a groupby aggregation](#Replicating-pivot_table-with-a-groupby-aggregation)
* [Renaming axis levels for easy reshaping](#Renaming-axis-levels-for-easy-reshaping)
* [Tidying when multiple variables are stored as column names](#Tidying-when-multiple-variables-are-stored-as-column-names)
* [Tidying when multiple variables are stored as column values](#Tidying-when-multiple-variables-are-stored-as-column-values)
* [Tidying when two or more values are stored in the same cell](#Tidying-when-two-or-more-values-are-stored-in-the-same-cell)
* [Tidying when variables are stored in column names and values](#Tidying-when-variables-are-stored-in-column-names-and-values)
* [Tidying when multiple observational units are stored in the same table](#Tidying-when-multiple-observational-units-are-stored-in-the-same-table)

In [ ]:
import pandas as pd
import numpy as np

# Tidying variable values as column names with stack

In [ ]:
state_fruit = pd.read_csv('data/state_fruit.csv', index_col=0)
state_fruit

In [ ]:
state_fruit.stack()

In [ ]:
state_fruit_tidy = state_fruit.stack().reset_index()
state_fruit_tidy

In [ ]:
state_fruit_tidy.columns = ['state', 'fruit', 'weight']
state_fruit_tidy

In [ ]:
state_fruit.stack()\
           .rename_axis(['state', 'fruit'])\

In [ ]:
state_fruit.stack()\
           .rename_axis(['state', 'fruit'])\
           .reset_index(name='weight')

## There's more...

In [ ]:
state_fruit2 = pd.read_csv('data/state_fruit2.csv')
state_fruit2

In [ ]:
state_fruit2.stack()

In [ ]:
state_fruit2.set_index('State').stack()

# Tidying variable values as column names with melt

In [ ]:
state_fruit2 = pd.read_csv('data/state_fruit2.csv')
state_fruit2

In [ ]:
state_fruit2.melt(id_vars=['State'],
                 value_vars=['Apple', 'Orange', 'Banana'])

In [ ]:
state_fruit2.index=list('abc')
state_fruit2.index.name = 'letter'

In [ ]:
state_fruit2

In [ ]:
state_fruit2.melt(id_vars=['State'],
                 value_vars=['Apple', 'Orange', 'Banana'],
                 var_name='Fruit',
                 value_name='Weight')

## There's more...

In [ ]:
state_fruit2.melt()

In [ ]:
state_fruit2.melt(id_vars='State')

# Stacking multiple groups of variables simultaneously

In [ ]:
movie = pd.read_csv('data/movie.csv')
actor = movie[['movie_title', 'actor_1_name', 'actor_2_name', 'actor_3_name', 
               'actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes']]
actor.head()

In [ ]:
def change_col_name(col_name):
    col_name = col_name.replace('_name', '')
    if 'facebook' in col_name:
        fb_idx = col_name.find('facebook')
        col_name = col_name[:5] + col_name[fb_idx - 1:] + col_name[5:fb_idx-1]
    return col_name

In [ ]:
actor2 = actor.rename(columns=change_col_name)
actor2.head()

In [ ]:
stubs = ['actor', 'actor_facebook_likes']
actor2_tidy = pd.wide_to_long(actor2, 
                              stubnames=stubs, 
                              i=['movie_title'], 
                              j='actor_num', 
                              sep='_').reset_index()
actor2_tidy.head()

## There's more...

In [ ]:
df = pd.read_csv('data/stackme.csv')
df

In [ ]:
df2 = df.rename(columns = {'a1':'group1_a1', 'b2':'group1_b2',
                           'd':'group2_a1', 'e':'group2_b2'})
df2

In [ ]:
pd.wide_to_long(df2, 
                stubnames=['group1', 'group2'], 
                i=['State', 'Country', 'Test'], 
                j='Label', 
                suffix='.+', 
                sep='_')

# Inverting stacked data

In [ ]:
usecol_func = lambda x: 'UGDS_' in x or x == 'INSTNM'
college = pd.read_csv('data/college.csv', 
                          index_col='INSTNM', 
                          usecols=usecol_func)
college.head()

In [ ]:
college_stacked = college.stack()
college_stacked.head(18)

In [ ]:
college_stacked.unstack().head()

In [ ]:
college2 = pd.read_csv('data/college.csv', 
                      usecols=usecol_func)
college2.head()

In [ ]:
college_melted = college2.melt(id_vars='INSTNM', 
                               var_name='Race',
                               value_name='Percentage')
college_melted.head()

In [ ]:
melted_inv = college_melted.pivot(index='INSTNM',
                                  columns='Race',
                                  values='Percentage')
melted_inv.head()

In [ ]:
college2_replication = melted_inv.loc[college2['INSTNM'], 
                                      college2.columns[1:]]\
                                         .reset_index()
college2.equals(college2_replication)

## There's more...

In [ ]:
college.stack().unstack(0)

In [ ]:
college.T

# Unstacking after a groupby aggregation

In [ ]:
employee = pd.read_csv('data/employee.csv')

In [ ]:
employee.groupby('RACE')['BASE_SALARY'].mean().astype(int)

In [ ]:
agg = employee.groupby(['RACE', 'GENDER'])['BASE_SALARY'].mean().astype(int)
agg

In [ ]:
agg.unstack('GENDER')

In [ ]:
agg.unstack('RACE')

## There's more...

In [ ]:
agg2 = employee.groupby(['RACE', 'GENDER'])['BASE_SALARY'].agg(['mean', 'max', 'min']).astype(int)
agg2

# Replicating pivot_table with a groupby aggregation

In [ ]:
flights = pd.read_csv('data/flights.csv')
flights.head()

In [ ]:
fp = flights.pivot_table(index='AIRLINE', 
                         columns='ORG_AIR', 
                         values='CANCELLED', 
                         aggfunc='sum',
                         fill_value=0).round(2)
fp.head()

In [ ]:
fg = flights.groupby(['AIRLINE', 'ORG_AIR'])['CANCELLED'].sum()
fg.head()

In [ ]:
fg_unstack = fg.unstack('ORG_AIR', fill_value=0)
fg_unstack.head()

In [ ]:
fp.equals(fg_unstack)

## There's more...

In [ ]:
fp2 = flights.pivot_table(index=['AIRLINE', 'MONTH'],
                          columns=['ORG_AIR', 'CANCELLED'],
                          values=['DEP_DELAY', 'DIST'],
                          aggfunc=[np.mean, np.sum],
                          fill_value=0)
fp2.head()

In [ ]:
flights.groupby(['AIRLINE', 'MONTH', 'ORG_AIR', 'CANCELLED'])['DEP_DELAY', 'DIST'] \
       .agg(['mean', 'sum']) \
       .unstack(['ORG_AIR', 'CANCELLED'], fill_value=0) \
       .swaplevel(0, 1, axis='columns') \
       .head()

# Renaming axis levels for easy reshaping

In [ ]:
college = pd.read_csv('data/college.csv')

In [ ]:
cg = college.groupby(['STABBR', 'RELAFFIL'])['UGDS', 'SATMTMID'] \
            .agg(['count', 'min', 'max']).head(6)

In [ ]:
cg

In [ ]:
cg = cg.rename_axis(['AGG_COLS', 'AGG_FUNCS'], axis='columns')
cg

In [ ]:
cg.stack('AGG_FUNCS').head()

In [ ]:
cg.stack('AGG_FUNCS').swaplevel('AGG_FUNCS', 'STABBR', axis='index').head()

In [ ]:
cg.stack('AGG_FUNCS') \
  .swaplevel('AGG_FUNCS', 'STABBR', axis='index') \
  .sort_index(level='RELAFFIL', axis='index') \
  .sort_index(level='AGG_COLS', axis='columns').head(6)

In [ ]:
cg.stack('AGG_FUNCS').unstack(['RELAFFIL', 'STABBR'])

In [ ]:
cg.stack(['AGG_FUNCS', 'AGG_COLS']).head(12)

# There's more...

In [ ]:
cg.rename_axis([None, None], axis='index').rename_axis([None, None], axis='columns')

# Tidying when multiple variables are stored as column names

In [ ]:
weightlifting = pd.read_csv('data/weightlifting_men.csv')
weightlifting

In [ ]:
wl_melt = weightlifting.melt(id_vars='Weight Category', 
                             var_name='sex_age', 
                             value_name='Qual Total')
wl_melt.head()

In [ ]:
sex_age = wl_melt['sex_age'].str.split(expand=True)
sex_age.head()

In [ ]:
sex_age.columns = ['Sex', 'Age Group']
sex_age.head()

In [ ]:
sex_age['Sex'] = sex_age['Sex'].str[0]
sex_age.head()

In [ ]:
wl_cat_total = wl_melt[['Weight Category', 'Qual Total']]
wl_tidy = pd.concat([sex_age, wl_cat_total], axis='columns')
wl_tidy.head()

In [ ]:
cols = ['Weight Category', 'Qual Total']
sex_age[cols] = wl_melt[cols]

## There's more...

In [ ]:
age_group = wl_melt.sex_age.str.extract('(\d{2}[-+](?:\d{2})?)', expand=False)
sex = wl_melt.sex_age.str[0]
new_cols = {'Sex':sex, 
            'Age Group': age_group}

In [ ]:
wl_tidy2 = wl_melt.assign(**new_cols).drop('sex_age', axis='columns')
wl_tidy2.head()

In [ ]:
wl_tidy2.sort_index(axis=1).equals(wl_tidy.sort_index(axis=1))

# Tidying when multiple variables are stored as column values

In [ ]:
inspections = pd.read_csv('data/restaurant_inspections.csv', parse_dates=['Date'])
inspections.head(10)

In [ ]:
inspections.pivot(index=['Name', 'Date'], columns='Info', values='Value')

In [ ]:
inspections.set_index(['Name','Date', 'Info']).head(10)

In [ ]:
inspections.set_index(['Name','Date', 'Info']).unstack('Info').head()

In [ ]:
insp_tidy = inspections.set_index(['Name','Date', 'Info']) \
                               .unstack('Info') \
                               .reset_index(col_level=-1)
insp_tidy.head()

In [ ]:
insp_tidy.columns = insp_tidy.columns.droplevel(0).rename(None)
insp_tidy.head()

In [ ]:
inspections.set_index(['Name','Date', 'Info']) \
          .squeeze() \
          .unstack('Info') \
          .reset_index() \
          .rename_axis(None, axis='columns')

## There's more...

In [ ]:
inspections.pivot_table(index=['Name', 'Date'], 
                        columns='Info', 
                        values='Value', 
                        aggfunc='first') \
           .reset_index()\
           .rename_axis(None, axis='columns')

# Tidying when two or more values are stored in the same cell

In [ ]:
cities = pd.read_csv('data/texas_cities.csv')
cities

In [ ]:
geolocations = cities.Geolocation.str.split(pat='. ', expand=True)
geolocations.columns = ['latitude', 'latitude direction', 'longitude', 'longitude direction']
geolocations

In [ ]:
geolocations = geolocations.astype({'latitude':'float', 'longitude':'float'})
geolocations.dtypes

In [ ]:
cities_tidy = pd.concat([cities['City'], geolocations], axis='columns')
cities_tidy

In [ ]:
pd.concat([cities['City'], geolocations], axis='columns')

## How it works...

In [ ]:
temp = geolocations.apply(pd.to_numeric, errors='ignore')
temp

In [ ]:
temp.dtypes

## There's more...

In [ ]:
cities.Geolocation.str.split(pat='° |, ', expand=True)

In [ ]:
cities.Geolocation.str.extract('([0-9.]+). (N|S), ([0-9.]+). (E|W)', expand=True)

# Tidying when variables are stored in column names and values

In [ ]:
sensors = pd.read_csv('data/sensors.csv')
sensors

In [ ]:
sensors.melt(id_vars=['Group', 'Property'], var_name='Year').head(6)

In [ ]:
sensors.melt(id_vars=['Group', 'Property'], var_name='Year') \
       .pivot_table(index=['Group', 'Year'], columns='Property', values='value') \
       .reset_index() \
       .rename_axis(None, axis='columns')

## There's more...

In [ ]:
sensors.set_index(['Group', 'Property']) \
       .stack() \
       .unstack('Property') \
       .rename_axis(['Group', 'Year'], axis='index') \
       .rename_axis(None, axis='columns') \
       .reset_index()

# Tidying when multiple observational units are stored in the same table

In [ ]:
movie = pd.read_csv('data/movie_altered.csv')
movie.head()

In [ ]:
movie.insert(0, 'id', np.arange(len(movie)))
movie.head()

In [ ]:
stubnames = ['director', 'director_fb_likes', 'actor', 'actor_fb_likes']
movie_long = pd.wide_to_long(movie, 
                                 stubnames=stubnames, 
                                 i='id', 
                                 j='num', 
                                 sep='_').reset_index()
movie_long['num'] = movie_long['num'].astype(int)
movie_long.head(9)

In [ ]:
movie_table = movie_long[['id','title', 'year', 'duration', 'rating']]
director_table = movie_long[['id', 'director', 'num', 'director_fb_likes']]
actor_table = movie_long[['id', 'actor', 'num', 'actor_fb_likes']]

In [ ]:
movie_table.head(9)

In [ ]:
director_table.head(9)

In [ ]:
actor_table.head(9)

In [ ]:
movie_table = movie_table.drop_duplicates().reset_index(drop=True)
director_table = director_table.dropna().reset_index(drop=True)
actor_table = actor_table.dropna().reset_index(drop=True)

In [ ]:
movie_table.head()

In [ ]:
director_table.head()

In [ ]:
movie.memory_usage(deep=True).sum()

In [ ]:
movie_table.memory_usage(deep=True).sum() + \
director_table.memory_usage(deep=True).sum() + \
actor_table.memory_usage(deep=True).sum()

In [ ]:
director_cat = pd.Categorical(director_table['director'])
director_table.insert(1, 'director_id', director_cat.codes)

actor_cat = pd.Categorical(actor_table['actor'])
actor_table.insert(1, 'actor_id', actor_cat.codes)

director_table.head()

In [ ]:
actor_table.head()

In [ ]:
director_associative = director_table[['id', 'director_id', 'num']]
dcols = ['director_id', 'director', 'director_fb_likes']
director_unique = director_table[dcols].drop_duplicates().reset_index(drop=True)
director_associative.head()                     

In [ ]:
director_unique.head()

In [ ]:
actor_associative = actor_table[['id', 'actor_id', 'num']]
acols = ['actor_id', 'actor', 'actor_fb_likes']
actor_unique = actor_table[acols].drop_duplicates().reset_index(drop=True)
actor_associative.head()

In [ ]:
actor_unique.head()

In [ ]:
movie_table.memory_usage(deep=True).sum() + \
director_associative.memory_usage(deep=True).sum() + \
director_unique.memory_usage(deep=True).sum() + \
actor_associative.memory_usage(deep=True).sum() + \
actor_unique.memory_usage(deep=True).sum()

In [ ]:
movie_table.head()

In [ ]:
actors = actor_associative.merge(actor_unique, on='actor_id') \
                          .drop('actor_id', 1) \
                          .pivot_table(index='id', columns='num', aggfunc='first')

actors.columns = actors.columns.get_level_values(0) + '_' + \
                 actors.columns.get_level_values(1).astype(str)

directors = director_associative.merge(director_unique, on='director_id') \
                          .drop('director_id', 1) \
                          .pivot_table(index='id', columns='num', aggfunc='first')

directors.columns = directors.columns.get_level_values(0) + '_' + \
                    directors.columns.get_level_values(1).astype(str)

In [ ]:
actors.head()

In [ ]:
directors.head()

In [ ]:
movie2 = movie_table.merge(directors.reset_index(), on='id', how='left') \
                    .merge(actors.reset_index(), on='id', how='left')

In [ ]:
movie2.head()

In [ ]:
movie.equals(movie2[movie.columns])